---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
def blight_model():
    """
    
    """
    
    # Import relevant libraries
    import pandas as pd
    import numpy as np
    from datetime import datetime
    from sklearn.preprocessing import LabelEncoder
    from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.metrics import roc_auc_score, roc_curve, auc
    from sklearn.naive_bayes import GaussianNB
    from sklearn.svm import SVC
    
    # Read dataframes
    # I included the argument dtype on the training dataset to avoid a DtypeWarning on columns (11, 12, 31)
    train = pd.read_csv('readonly/train.csv', encoding='cp1252', 
                           dtype = {'zip_code': object, 'non_us_str_code': object, 'grafitti_status': object})
    test = pd.read_csv('readonly/test.csv', encoding='cp1252')
    address = pd.read_csv('readonly/addresses.csv')
    latlong = pd.read_csv('readonly/latlons.csv')
    
    # Join address and latlong in one dataframe.  Then, join address to both train and test data
    address = address.set_index('address').join(latlong.set_index('address'), how = 'left')
    train = train.set_index('ticket_id').join(address.set_index('ticket_id'), how = 'left')
    test = test.set_index('ticket_id').join(address.set_index('ticket_id'), how = 'left')
    
    # Drop NaN on target value 'compliance' --> we are just looking for compliants and or not compliants
    # Although the target variable 'compliance' is dtype('float64'). So, I will adjust it
    # to meake it categorical
    train = train.dropna(subset = ['compliance'])
    train['compliance'] = train['compliance'].astype(int)
    
    # I created a spreadsheet in which I stored the features I wanted to use on the training and
    # and testing dataframes.  Then, I created the mask from there.
    #mask = pd.read_excel('model_columns_final.xlsx')
    #mask = mask['columnas']
    #mask = mask.tolist()
    
    # I commented out this part of the code as the autograder will get an error from it.
    # Instead, I just created a variable called "mask", which I used to keep all the attributions 
    # I will use on this excercise.
    
    # Create the mask variables with all the variables you want to use
    mask = ['hearing_date', 
            'ticket_issued_date', 
            'discount_amount',
            'disposition', 
            'fine_amount', 
            'judgment_amount', 
            'late_fee', 
            'state', 
            'violation_code', 
            'lat',
            'lon',
            'compliance']
    
    # Create the final training dataframe and adjust its data types as well as create a new variable 
    # named "date_diff"
    #train = train [mask].dropna()
    train['hearing_date'] = pd.to_datetime(train['hearing_date'], errors = 'coerce')
    train['hearing_date'] = train['hearing_date'].fillna(method = 'pad')
    train['ticket_issued_date'] = pd.to_datetime(train['ticket_issued_date'], errors = 'coerce')
    train['ticket_issued_date'] = train['ticket_issued_date'].fillna(method = 'pad')
    train['date_diff'] = (train['hearing_date'] - train['ticket_issued_date'])
    train['date_diff'] = train['date_diff'].dt.days
    train = train[~train['date_diff'] < 0]
    
    # Create the final test dataframe and adjust its data types as well as create a new variable 
    # named "date_diff"
    mask.remove('compliance')
    test = test[mask].dropna()
    test['hearing_date'] = pd.to_datetime(test['hearing_date'], errors = 'coerce')
    test['hearing_date'] = test['hearing_date'].fillna(method = 'pad')
    test['ticket_issued_date'] = pd.to_datetime(test['ticket_issued_date'], errors = 'coerce')
    test['ticket_issued_date'] = test['ticket_issued_date'].fillna(method = 'pad')
    test['date_diff'] = (test['hearing_date'] - test['ticket_issued_date'])
    test['date_diff'] = test['date_diff'].dt.days
    test = test[~test['date_diff'] < 0]
    
    
    
    '''
    As a side note, I wanted to document my decision making regarding the attributions
    I ended up taking
    
    1.- By the moment I was selecting the variables, I wanted to add "zip_code" attribution but 
    I noticed that there were 1048 rows with wrong-formatted in the training data.  This is what I got:
    
        5.0     158831
        4.0        638
        6.0        138
        1.0        107
        3.0         60
        10.0        54
        9.0         29
        2.0         12
        7.0          7
        8.0          3
        Name: zip, dtype: int64
    
    The "zip" attribution I created, measured the lenght of each zip code
    
    '''
    
    
    
    
    
    
    
    return test.head(2)

blight_model()

,hearing_date,ticket_issued_date,discount_amount,disposition,fine_amount,judgment_amount,late_fee,state,violation_code,lat,lon,date_diff
ticket_id,,,,,,,,,,,,
284932,2012-01-19 09:00:00,2012-01-04 14:00:00,0.0,Responsible by Default,200.0,250.0,20.0,MI,22-2-61,42.407581,-82.986642,14
285362,2012-02-06 09:00:00,2012-01-05 09:50:00,0.0,Responsible by Default,1000.0,1130.0,100.0,MI,22-2-88(b),42.426239,-83.238259,31


In [2]:
blight_model()

,hearing_date,ticket_issued_date,discount_amount,disposition,fine_amount,judgment_amount,late_fee,state,violation_code,lat,lon,date_diff
ticket_id,,,,,,,,,,,,
284932,2012-01-19 09:00:00,2012-01-04 14:00:00,0.0,Responsible by Default,200.0,250.0,20.0,MI,22-2-61,42.407581,-82.986642,14
285362,2012-02-06 09:00:00,2012-01-05 09:50:00,0.0,Responsible by Default,1000.0,1130.0,100.0,MI,22-2-88(b),42.426239,-83.238259,31


In [3]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [4]:
df_train = pd.read_csv('readonly/train.csv', encoding= 'ISO-8859-1', dtype = {'zip_code': object, 'non_us_str_code': object, 'grafitti_status': object})
df_train.head(3)

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [5]:
df_train.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail', 'compliance'],
      dtype='object')

In [6]:
df_train.shape

(250306, 34)

In [7]:
df_train.describe()

,ticket_id,violation_street_number,violation_zip_code,mailing_address_str_number,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,payment_amount,balance_due,compliance
count,250306.000000,2.503060e+05,0.0,2.467040e+05,250305.000000,250306.000000,250306.000000,250306.000000,250306.000000,250306.0,250306.000000,250306.000000,250306.000000,159880.000000
mean,152665.543099,1.064986e+04,NaN,9.149788e+03,374.423435,12.774764,6.387382,21.494506,0.125167,0.0,268.685356,48.898986,222.449058,0.072536
std,77189.882881,3.188733e+04,NaN,3.602034e+04,707.195807,9.607344,4.803672,56.464263,3.430178,0.0,626.915212,222.422425,606.394010,0.259374
min,18645.000000,0.000000e+00,NaN,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-7750.000000,0.000000
25%,86549.250000,4.739000e+03,NaN,5.440000e+02,200.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,152597.500000,1.024400e+04,NaN,2.456000e+03,250.000000,20.000000,10.000000,10.000000,0.000000,0.0,140.000000,0.000000,25.000000,0.000000
75%,219888.750000,1.576000e+04,NaN,1.292725e+04,250.000000,20.000000,10.000000,25.000000,0.000000,0.0,305.000000,0.000000,305.000000,0.000000
max,366178.000000,1.415411e+07,NaN,5.111345e+06,10000.000000,20.000000,10.000000,1000.000000,350.000000,0.0,11030.000000,11075.000000,11030.000000,1.000000


In [8]:
df_test = pd.read_csv('readonly/test.csv', encoding= 'ISO-8859-1')
df_test.head(3)

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN


In [9]:
df_test.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status'],
      dtype='object')

In [10]:
df_test.shape

(61001, 27)

In [11]:
df_test.describe()

,ticket_id,violation_street_number,non_us_str_code,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount
count,61001.000000,6.100100e+04,0.0,61001.000000,61001.0,61001.0,61001.000000,61001.000000,61001.000000,61001.000000
mean,331724.532811,1.256638e+04,NaN,272.714185,20.0,10.0,25.116219,0.239340,20.649711,347.895541
std,25434.932141,1.414373e+05,NaN,360.101855,0.0,0.0,36.310155,3.245894,242.375180,460.058043
min,284932.000000,-1.512600e+04,NaN,0.000000,20.0,10.0,0.000000,0.000000,0.000000,0.000000
25%,310111.000000,6.008000e+03,NaN,50.000000,20.0,10.0,5.000000,0.000000,0.000000,85.000000
50%,332251.000000,1.213400e+04,NaN,200.000000,20.0,10.0,10.000000,0.000000,0.000000,250.000000
75%,353031.000000,1.716500e+04,NaN,250.000000,20.0,10.0,25.000000,0.000000,0.000000,305.000000
max,376698.000000,2.010611e+07,NaN,10000.000000,20.0,10.0,1000.000000,250.000000,15309.000000,15558.800000


let's remember that the target variable is Compliance.  I want to know how many NaN happens in the target variable

In [12]:
address = pd.read_csv('readonly/addresses.csv')
address.head()

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [13]:
address.shape

(311307, 2)

In [14]:
latlong = pd.read_csv('readonly/latlons.csv')
latlong.head()

,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295


In [15]:
latlong.shape

(121769, 3)

In [16]:
address = address.set_index('address').join(latlong.set_index('address'), how = 'left')

In [17]:
df_train = df_train.dropna(subset = ['compliance'])
df_train['compliance'] = df_train['compliance'].astype(int)
df_train.head()


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1
5,22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0
6,18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,NaN,476.0,Garfield,Clinton,...,0.0,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0
7,18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,...,0.0,140.0,0.0,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0


In [18]:
df_train = df_train.set_index('ticket_id').join(address.set_index('ticket_id'), how = 'left')
df_train.head(3)

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,
22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,IL,...,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0,42.390729,-83.124268
27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,MI,...,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1,42.326937,-83.135118
22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,CA,...,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0,42.145257,-83.208233


In [19]:
df_test = df_test.set_index('ticket_id').join(address.set_index('ticket_id'), how = 'left')
df_test.head()

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,
284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,MI,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,42.407581,-82.986642
285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN,42.426239,-83.238259
285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,42.426239,-83.238259
285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,MI,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,42.309661,-83.122426
285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,MI,...,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,42.308830,-83.121116


In [20]:
df_train.shape

(159880, 35)

In [21]:
mask = pd.read_excel('model_columns.xlsx')
mask = mask['columnas']
mask = mask.tolist()
mask

['hearing_date',
 'ticket_issued_date',
 'clean_up_cost',
 'country',
 'discount_amount',
 'disposition',
 'fine_amount',
 'judgment_amount',
 'late_fee',
 'state',
 'violation_code',
 'zip_code',
 'lat',
 'lon',
 'compliance']

In [22]:
df_train_mesmo = df_train[mask]
df_train_mesmo.head()

,hearing_date,ticket_issued_date,clean_up_cost,country,discount_amount,disposition,fine_amount,judgment_amount,late_fee,state,violation_code,zip_code,lat,lon,compliance
ticket_id,,,,,,,,,,,,,,,
22056,2005-03-21 10:30:00,2004-03-16 11:40:00,0.0,USA,0.0,Responsible by Default,250.0,305.0,25.0,IL,9-1-36(a),60606,42.390729,-83.124268,0
27586,2005-05-06 13:30:00,2004-04-23 12:30:00,0.0,USA,0.0,Responsible by Determination,750.0,855.0,75.0,MI,61-63.0600,48208,42.326937,-83.135118,1
22046,2005-03-21 10:30:00,2004-05-01 11:50:00,0.0,USA,0.0,Responsible by Default,250.0,305.0,25.0,CA,9-1-36(a),908041512,42.145257,-83.208233,0
18738,2005-02-22 15:00:00,2004-06-14 14:15:00,0.0,USA,0.0,Responsible by Default,750.0,855.0,75.0,MI,61-63.0500,48038,42.433466,-83.023493,0
18735,2005-02-22 15:00:00,2004-06-16 12:30:00,0.0,USA,0.0,Responsible by Default,100.0,140.0,10.0,MI,61-63.0100,48211,42.388641,-83.037858,0


In [23]:
#df_train_mesmo.loc[:,'hearing_date'] = df_train_mesmo.loc[:,'hearing_date'].astype('datetime64[ns]')
#df_train_mesmo['ticket_issued_date'] = df_train_mesmo['ticket_issued_date'].astype('datetime')

df_train_mesmo.to_csv('train_prueba.csv')

In [24]:
# At this point I have made so many changes that I just wanted to confirm that I successfully
# got rid of the empty "compliance" rows. I got 159880, which is less than the original 
# 250306 original rows
df_train.shape

(159880, 35)

Once we have a clear idea about the attributions we are going to use, let's work with our dates

In [25]:

#df_train_mesmo['hearing_date']  = df_train_mesmo['hearing_date'].apply(str)

#df_train_mesmo['hearing_date'] = pd.datetime(df_train_mesmo['hearing_date'],format = "%Y-%m-%d %H:%M:%S")
#df_train_mesmo['hearing'] = df_train_mesmo.loc[:,'hearing_date'].copy()
#df_train_mesmo['hearing'] = df_train_mesmo.loc[:,'hearing_date'].apply(str)

#df_train_mesmo.loc[:,'hearing_date'] = df_train_mesmo.hearing_date.copy()
#df_train_mesmo.hearing_date.apply(type).value_counts()
#df_train_mesmo['hearing_date'] = df_train_mesmo.hearing_date.copy(deep=True)


#df_train_mesmo.loc[:, 'hearing_date'] = df_train_mesmo.hearing_date.copy(deep=True)
#df_train_mesmo.hearing_date = df_train_mesmo.hearing_date.fillna('2019-02-01 00:00:00')


#df_train_mesmo = df_train_mesmo.dropna()
#df_train_mesmo.shape
#df_train_mesmo['diff'] = pd.to_datetime(df_train_mesmo['hearing_date'], format = "%Y-%m-%d %H:%M:%S") - pd.to_datetime(df_train_mesmo['ticket_issued_date'], format = "%Y-%m-%d %H:%M:%S")
import datetime as dt

#df_train_mesmo['hearing_date'] = pd.to_datetime(df_train_mesmo['hearing_date']).dt.datetime
#df_train_mesmo['hearing_date'] = pd.to_datetime(df_train_mesmo['hearing_date']).dt.values.astype(np.int64)
df_train_mesmo['hearing_date'] = pd.to_datetime(df_train_mesmo['hearing_date'], errors = 'coerce')
df_train_mesmo['ticket_issued_date'] = pd.to_datetime(df_train_mesmo['ticket_issued_date'], errors = 'coerce')
df_train_mesmo['hearing_date'].head()


#df_train_mesmo.to_csv('fechas_mesmas.csv')
df_train_mesmo.to_csv('fechas_dropna.csv')
df_train_mesmo[['hearing_date', 'ticket_issued_date']].head()
df_train_mesmo['diff'] = df_train_mesmo['hearing_date'] - df_train_mesmo['ticket_issued_date']
df_train_mesmo['diff'] #.dt.values.astype(np.int64)

C:\Users\JohnHill\Anaconda3\lib\site-packages\ipykernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\JohnHill\Anaconda3\lib\site-packages\ipykernel\__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\JohnHill\Anaconda3\lib\site-packages\ipykernel\__main__.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

ticket_id
22056      369 days 22:50:00
27586      378 days 01:00:00
22046      323 days 22:40:00
18738      253 days 00:45:00
18735      251 days 02:30:00
18733      251 days 02:35:00
28204      323 days 00:00:00
18743      209 days 02:45:00
18741      201 days 00:00:00
18978      189 days 00:45:00
18746      138 days 03:00:00
18744      138 days 03:00:00
26846      190 days 10:45:00
26848      189 days 23:15:00
28209      215 days 02:30:00
19950       40 days 04:00:00
18645       24 days 19:00:00
18651       30 days 18:35:00
18649       30 days 18:10:00
18664       68 days 05:30:00
18646       31 days 08:15:00
18661       35 days 04:45:00
18657       35 days 05:00:00
18652       30 days 17:50:00
18665       35 days 04:25:00
18650       30 days 18:55:00
18653       39 days 17:45:00
18658       21 days 03:30:00
18666       35 days 04:20:00
18655       30 days 17:20:00
                 ...        
268081   -280 days +06:00:00
267964   -224 days +06:00:00
267966   -295 days +06:00:00
2679

In [26]:
import datetime as dt

df_train_mesmo['hearing_date'] = pd.to_datetime(df_train_mesmo['hearing_date'], errors = 'coerce')
df_train_mesmo['hearing_date'] = df_train_mesmo['hearing_date'].fillna( method = 'pad')
#df_train_mesmo['ticket_issued_date'] = pd.to_datetime(df_train_mesmo['ticket_issued_date'], errors = 'coerce')
df_train_mesmo['hearing_date'].apply(type).value_counts()

C:\Users\JohnHill\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\JohnHill\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<class 'pandas._libs.tslibs.timestamps.Timestamp'>    159880
Name: hearing_date, dtype: int64

In [27]:
df_train_mesmo[df_train_mesmo['hearing_date'].isnull()]

,hearing_date,ticket_issued_date,clean_up_cost,country,discount_amount,disposition,fine_amount,judgment_amount,late_fee,state,violation_code,zip_code,lat,lon,compliance,diff
ticket_id,,,,,,,,,,,,,,,,


In [28]:
import datetime as dt

df_train_mesmo['hearing_date'] = pd.to_datetime(df_train_mesmo['hearing_date'], errors = 'coerce')
df_train_mesmo['ticket_issued_date'] = pd.to_datetime(df_train_mesmo['ticket_issued_date'], errors = 'coerce')
df_train_mesmo['hearing_date'].head()

C:\Users\JohnHill\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\JohnHill\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


ticket_id
22056   2005-03-21 10:30:00
27586   2005-05-06 13:30:00
22046   2005-03-21 10:30:00
18738   2005-02-22 15:00:00
18735   2005-02-22 15:00:00
Name: hearing_date, dtype: datetime64[ns]

In [29]:
df_train_mesmo.hearing_date.fillna('2019-02-01 00:00:00').apply(type).value_counts()


<class 'pandas._libs.tslibs.timestamps.Timestamp'>    159880
Name: hearing_date, dtype: int64

#### Let's prepare the df_train

Let's use directly on the df_train

In [30]:
mask = pd.read_excel('model_columns_final.xlsx')
mask = mask['columnas']
mask = mask.tolist()
mask

['hearing_date',
 'ticket_issued_date',
 'discount_amount',
 'disposition',
 'fine_amount',
 'judgment_amount',
 'late_fee',
 'state',
 'violation_code',
 'lat',
 'lon',
 'compliance']

In [31]:
df_train = df_train[mask].dropna()
df_train.head()

,hearing_date,ticket_issued_date,discount_amount,disposition,fine_amount,judgment_amount,late_fee,state,violation_code,lat,lon,compliance
ticket_id,,,,,,,,,,,,
22056,2005-03-21 10:30:00,2004-03-16 11:40:00,0.0,Responsible by Default,250.0,305.0,25.0,IL,9-1-36(a),42.390729,-83.124268,0
27586,2005-05-06 13:30:00,2004-04-23 12:30:00,0.0,Responsible by Determination,750.0,855.0,75.0,MI,61-63.0600,42.326937,-83.135118,1
22046,2005-03-21 10:30:00,2004-05-01 11:50:00,0.0,Responsible by Default,250.0,305.0,25.0,CA,9-1-36(a),42.145257,-83.208233,0
18738,2005-02-22 15:00:00,2004-06-14 14:15:00,0.0,Responsible by Default,750.0,855.0,75.0,MI,61-63.0500,42.433466,-83.023493,0
18735,2005-02-22 15:00:00,2004-06-16 12:30:00,0.0,Responsible by Default,100.0,140.0,10.0,MI,61-63.0100,42.388641,-83.037858,0


In [32]:
df_train.shape

(159567, 12)

In [33]:
df_train.hearing_date.dtype

dtype('O')

In [34]:
import datetime as dt
df_train['hearing_date'] = pd.to_datetime(df_train['hearing_date'], format = "%Y-%m-%d") 
df_train['hearing_date']

ticket_id
22056    2005-03-21 10:30:00
27586    2005-05-06 13:30:00
22046    2005-03-21 10:30:00
18738    2005-02-22 15:00:00
18735    2005-02-22 15:00:00
18733    2005-02-22 15:00:00
28204    2005-05-31 13:30:00
18743    2005-02-22 15:00:00
18741    2005-02-22 15:00:00
18978    2005-02-28 15:00:00
18746    2005-02-22 15:00:00
18744    2005-02-22 15:00:00
26846    2005-04-21 13:30:00
26848    2005-04-21 13:30:00
28209    2005-05-31 13:30:00
19950    2005-02-10 15:00:00
18645    2005-02-07 09:00:00
18651    2005-02-14 09:00:00
18649    2005-02-14 09:00:00
18664    2005-03-23 15:00:00
18646    2005-02-14 09:00:00
18661    2005-02-18 15:00:00
18657    2005-02-18 15:00:00
18652    2005-02-14 09:00:00
18665    2005-02-18 15:00:00
18650    2005-02-14 09:00:00
18653    2005-02-23 09:00:00
18658    2005-02-04 13:30:00
18666    2005-02-18 15:00:00
18655    2005-02-14 09:00:00
                 ...        
268081   2011-03-17 15:00:00
267964   2011-05-12 15:00:00
267966   2011-03-02 15:00:00
2679

In [35]:
df_train.hearing_date.dtype

dtype('<M8[ns]')

In [36]:
from datetime import datetime
df_train['ticket_issued_date'] = pd.to_datetime(df_train['ticket_issued_date'],format = "%Y-%m-%d %H:%M:%S")
df_train['ticket_issued_date']

ticket_id
22056    2004-03-16 11:40:00
27586    2004-04-23 12:30:00
22046    2004-05-01 11:50:00
18738    2004-06-14 14:15:00
18735    2004-06-16 12:30:00
18733    2004-06-16 12:25:00
28204    2004-07-12 13:30:00
18743    2004-07-28 12:15:00
18741    2004-08-05 15:00:00
18978    2004-08-23 14:15:00
18746    2004-10-07 12:00:00
18744    2004-10-07 12:00:00
26846    2004-10-13 02:45:00
26848    2004-10-13 14:15:00
28209    2004-10-28 11:00:00
19950    2005-01-01 11:00:00
18645    2005-01-13 14:00:00
18651    2005-01-14 14:25:00
18649    2005-01-14 14:50:00
18664    2005-01-14 09:30:00
18646    2005-01-14 00:45:00
18661    2005-01-14 10:15:00
18657    2005-01-14 10:00:00
18652    2005-01-14 15:10:00
18665    2005-01-14 10:35:00
18650    2005-01-14 14:05:00
18653    2005-01-14 15:15:00
18658    2005-01-14 10:00:00
18666    2005-01-14 10:40:00
18655    2005-01-14 15:40:00
                 ...        
268081   2011-12-22 09:00:00
267964   2011-12-22 09:00:00
267966   2011-12-22 09:00:00
2679

In [37]:
import datetime as dt
df_train['date_diff'] = (df_train['hearing_date'] - df_train['ticket_issued_date'])
df_train['date_diff'] = df_train['date_diff'].dt.days
df_train['date_diff']

ticket_id
22056     369
27586     378
22046     323
18738     253
18735     251
18733     251
28204     323
18743     209
18741     201
18978     189
18746     138
18744     138
26846     190
26848     189
28209     215
19950      40
18645      24
18651      30
18649      30
18664      68
18646      31
18661      35
18657      35
18652      30
18665      35
18650      30
18653      39
18658      21
18666      35
18655      30
         ... 
268081   -280
267964   -224
267966   -295
267974   -280
267985   -280
267970   -294
284870     13
284873     19
284871     19
284875     19
284874     19
285091     30
285508     44
285093     30
285095     30
285121     37
285122     37
285120     37
285123     37
285092     30
285094     30
285096     30
285036     29
285037     29
285034     29
285106     37
284650     10
285125     26
284881     40
284333      5
Name: date_diff, Length: 159567, dtype: int64

In [38]:
df_train['date_diff'].dtype

dtype('int64')

In [39]:
df_train[~df_train['date_diff'] < 0].shape

(159487, 13)

In [40]:
df_train[~df_train['date_diff'] < 0]

,hearing_date,ticket_issued_date,discount_amount,disposition,fine_amount,judgment_amount,late_fee,state,violation_code,lat,lon,compliance,date_diff
ticket_id,,,,,,,,,,,,,
22056,2005-03-21 10:30:00,2004-03-16 11:40:00,0.0,Responsible by Default,250.0,305.0,25.0,IL,9-1-36(a),42.390729,-83.124268,0,369
27586,2005-05-06 13:30:00,2004-04-23 12:30:00,0.0,Responsible by Determination,750.0,855.0,75.0,MI,61-63.0600,42.326937,-83.135118,1,378
22046,2005-03-21 10:30:00,2004-05-01 11:50:00,0.0,Responsible by Default,250.0,305.0,25.0,CA,9-1-36(a),42.145257,-83.208233,0,323
18738,2005-02-22 15:00:00,2004-06-14 14:15:00,0.0,Responsible by Default,750.0,855.0,75.0,MI,61-63.0500,42.433466,-83.023493,0,253
18735,2005-02-22 15:00:00,2004-06-16 12:30:00,0.0,Responsible by Default,100.0,140.0,10.0,MI,61-63.0100,42.388641,-83.037858,0,251
18733,2005-02-22 15:00:00,2004-06-16 12:25:00,0.0,Responsible by Default,100.0,140.0,10.0,MI,61-63.0100,42.388641,-83.037858,0,251
28204,2005-05-31 13:30:00,2004-07-12 13:30:00,0.0,Responsible by Default,750.0,855.0,75.0,MI,61-63.0600,42.435773,-82.963348,0,323
18743,2005-02-22 15:00:00,2004-07-28 12:15:00,0.0,Responsible by Default,750.0,855.0,75.0,MI,61-63.0600,42.395765,-83.022333,0,209
18741,2005-02-22 15:00:00,2004-08-05 15:00:00,0.0,Responsible by Default,750.0,855.0,75.0,MI,61-63.0600,42.440190,-83.180488,0,201


In [41]:
mask = ['hearing_date',
 'ticket_issued_date',
 'clean_up_cost',
 'country',
 'discount_amount',
 'disposition',
 'fine_amount',
 'judgment_amount',
 'late_fee',
 'state',
 'violation_code',
 'zip_code',
 'lat',
 'lon',
 'compliance']
mask
mask.remove('compliance')
df_test = df_test[mask].dropna()
df_test.shape

(58470, 14)

In [42]:
mask = pd.read_excel('model_columns.xlsx')
mask = mask['columnas']
mask = mask.tolist()
mask

['hearing_date',
 'ticket_issued_date',
 'clean_up_cost',
 'country',
 'discount_amount',
 'disposition',
 'fine_amount',
 'judgment_amount',
 'late_fee',
 'state',
 'violation_code',
 'zip_code',
 'lat',
 'lon',
 'compliance']

In [43]:
'''
for df in [df_train, df_test]:
    if df == df_test:
        mask.remove('compliance')
    for col in mask:
        df[col] = df[col].astype('category')
'''        


"\nfor df in [df_train, df_test]:\n    if df == df_test:\n        mask.remove('compliance')\n    for col in mask:\n        df[col] = df[col].astype('category')\n"

In [44]:
convert_columns={'country': 'category',
                     'non_us_str_code': 'category',
                     'compliance': 'category',
                     'state': 'category',
                     'zip_code': 'category'
                    }

convert_columns.items()

dict_items([('state', 'category'), ('country', 'category'), ('non_us_str_code', 'category'), ('zip_code', 'category'), ('compliance', 'category')])

In [45]:

for col in mask:
    df[col] = df[col].astype('category')

NameError: name 'df' is not defined

In [ ]:
df_test.shape